In [ ]:
#| default_exp core

# core

In [ ]:
#|export
import os,shutil,subprocess,tempfile
from contextlib import contextmanager
from pathlib import Path

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export
class TmpDir:
    "Create temporary workspaces."
    def __init__(self): self.cwd,self._dir,self.path = None,None,None

    def new(self, subdir=''):
        "Create and `cd` to `subdir` under a temp dir."
        if self.cwd is None: self.cwd = Path.cwd()
        self.dir = Path(tempfile.mkdtemp())
        self.path = self.dir/subdir
        self.path.mkdir(exist_ok=True, parents=True)
        os.chdir(self.path)

    @property
    def dir(self): return self._dir
    @dir.setter
    def dir(self, o):
        "`rm` current `dir` and set a new one."
        if self._dir: shutil.rmtree(self._dir)
        self._dir = o

    def close(self):
        "`rm` current `dir` and `cd` to original `cwd`."
        self.dir = None
        os.chdir(self.cwd)

    @contextmanager
    def __call__(self, subdir=''):
        "Work in a temp dir then `cd` back to original `cwd`."
        try:
            self.new(subdir)
            yield self.path
        finally: self.close()

The main way to use `TmpDir` is through the exported object:

In [ ]:
#|exports
tmpdir = TmpDir()

We start out in our project dir:

In [ ]:
d0 = Path.cwd(); d0

Path('/Users/seem/code/sketch-tmpdir')

Switch to path `foo/bar` under a temp dir:

In [ ]:
tmpdir.new('foo/bar')
d1 = tmpdir.dir
Path.cwd()

Path('/private/var/folders/ft/0gnvc3ts5jz4ddqtttp6tjvm0000gn/T/tmp19y8l0h4/foo/bar')

If we switch again, the previous dir is removed:

In [ ]:
tmpdir.new('foo/bar')
assert not d1.exists()
d2 = tmpdir.dir
Path.cwd()

Path('/private/var/folders/ft/0gnvc3ts5jz4ddqtttp6tjvm0000gn/T/tmpnjqprlft/foo/bar')

Finally, revert to the original working directory, which also removes the remaining temporary directory:

In [ ]:
tmpdir.close()
assert not d2.exists()
test_eq(Path.cwd(), d0)

You can also use it as a context manager to automatically revert to the original working directory at the end:

In [ ]:
with tmpdir() as p:
    d3 = tmpdir.dir
    test_eq(Path.cwd().name, p.name)
assert not d3.exists()
test_eq(Path.cwd(), d0)

One use-case is to write executable documentation for code that interacts with its workspace. For example, suppose you were writing the `git_repo` function of a Python git interface. You could demonstrate it as follows.

---

In [ ]:
def git_repo():
    "Remote repo from git config."
    cmd = 'git config --get remote.origin.url'
    proc = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if proc.returncode: return
    return proc.stdout.strip().split('/', 1)[1].split('.')[0]

Let's initialise a minimal repo to demonstrate:

In [ ]:
tmpdir.new()

In [ ]:
%%sh
git init -q
git remote add origin git@github.com:my-user/my-repo.git

Get the repo name:

In [ ]:
test_eq(git_repo(), 'my-repo')

Returns `None` if you're not in a git repo:

In [ ]:
with tmpdir(): test_is(git_repo(), None)

---

And you're back to your original working directory:

In [ ]:
Path.cwd()

Path('/Users/seem/code/sketch-tmpdir')

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()